In [181]:
import chess
import re
import numpy as np
import io

game = '''
[Event "Rated Bullet tournament https://lichess.org/tournament/yc1WW2Ox"]
[Site "https://lichess.org/PpwPOZMq"]
[Date "2017.04.01"]
[Round "-"]
[White "Abbot"]
[Black "Costello"]
[Result "0-1"]
[UTCDate "2017.04.01"]
[UTCTime "11:32:01"]
[WhiteElo "2100"]
[BlackElo "2000"]
[WhiteRatingDiff "-4"]
[BlackRatingDiff "+1"]
[WhiteTitle "FM"]
[ECO "B30"]
[Opening "Sicilian Defense: Old Sicilian"]
[TimeControl "300+0"]
[Termination "Time forfeit"]

1. e4 { [%eval 0.17] [%clk 0:00:30] } 1... c5 { [%eval 0.19] [%clk 0:00:30] }
2. Nf3 { [%eval 0.25] [%clk 0:00:29] } 2... Nc6 { [%eval 0.33] [%clk 0:00:30] }
3. Bc4 { [%eval -0.13] [%clk 0:00:28] } 3... e6 { [%eval -0.04] [%clk 0:00:30] }
4. c3 { [%eval -0.4] [%clk 0:00:27] } 4... b5? { [%eval 1.18] [%clk 0:00:30] }
5. Bb3?! { [%eval 0.21] [%clk 0:00:26] } 5... c4 { [%eval 0.32] [%clk 0:00:29] }
6. Bc2 { [%eval 0.2] [%clk 0:00:25] } 6... a5 { [%eval 0.6] [%clk 0:00:29] }
7. d4 { [%eval 0.29] [%clk 0:00:23] } 7... cxd3 { [%eval 0.6] [%clk 0:00:27] }
8. Qxd3 { [%eval 0.12] [%clk 0:00:22] } 8... Nf6 { [%eval 0.52] [%clk 0:00:26] }
9. e5 { [%eval 0.39] [%clk 0:00:21] } 9... Nd5 { [%eval 0.45] [%clk 0:00:25] }
10. Bg5?! { [%eval -0.44] [%clk 0:00:18] } 10... Qc7 { [%eval -0.12] [%clk 0:00:23] }
11. Nbd2?? { [%eval -3.15] [%clk 0:00:14] } 11... h6 { [%eval -2.99] [%clk 0:00:23] }
12. Bh4 { [%eval -3.0] [%clk 0:00:11] } 12... Ba6? { [%eval -0.12] [%clk 0:00:23] }
13. b3?? { [%eval -4.14] [%clk 0:00:02] } 13... Nf4? { [%eval #-2] [%clk 0:00:21] } 0-1
'''

In [95]:
def get_ratings(game_string):

    white_rating = int(re.search(r'\[WhiteElo "(\d+)"\]', game).group(1))
    black_rating = int(re.search(r'\[BlackElo "(\d+)"\]', game).group(1))

    return white_rating, black_rating


White rating: 2100
Black rating: 2000


In [ ]:
import chess.pgn

datafile = 'data/lichess.pgn'

target_dir = 'data/training/'

scanned_games = 0
game_number=0

with open(datafile,'r') as f:
    while game_number<1000000:
        scanned_games+=1
        if scanned_games%100000==0:
            print("scanned games: ",scanned_games)
        game = chess.pgn.read_game(f)
        if game is None:
            break
        #check that the time control is 300+0 and that the moves have an evaluation

        n = game.next()

        if game.headers['TimeControl'] == '300+0' and n!=None and n.eval()!=None:
            #write the pgn to a file in the target directory using game_number.pgn
            print(game,file = open(target_dir + str(game_number) + '.pgn','w'))
            game_number+=1
            if game_number % 1000 == 0:
                print("written games: ",game_number)



written games:  1000
scanned games:  100000
written games:  2000
scanned games:  200000
written games:  3000
written games:  4000
scanned games:  300000
written games:  5000
written games:  6000
scanned games:  400000
written games:  7000
scanned games:  500000
written games:  8000
written games:  9000
scanned games:  600000
written games:  10000
scanned games:  700000
written games:  11000
scanned games:  800000
written games:  12000
written games:  13000
scanned games:  900000
written games:  14000
scanned games:  1000000
written games:  15000
scanned games:  1100000
written games:  16000
written games:  17000
scanned games:  1200000
written games:  18000
scanned games:  1300000
written games:  19000
written games:  20000
scanned games:  1400000
written games:  21000
scanned games:  1500000
written games:  22000
scanned games:  1600000
written games:  23000
written games:  24000
scanned games:  1700000
written games:  25000
scanned games:  1800000
written games:  26000
scanned games:

In [182]:
pgn = chess.pgn.read_game(io.StringIO(game))

In [197]:
import keras

class GameSequence(keras.utils.Sequence):
    def __init__(self, pgn_files, batch_size=32, shuffle=True):
        self.pgn_files = pgn_files #this is a list of pgn files as strings/paths
        self.batch_size = batch_size
        self.shuffle = shuffle

        if shuffle:
            np.random.shuffle(self.pgn_files)
    
    def __len__(self):
        return len(self.pgn_files) // self.batch_size
    
    def __getitem__(self, idx):
        batch_files = self.pgn_files[idx * self.batch_size:(idx + 1) * self.batch_size]
        X, y = [], []
        for file in batch_files:
            pgn = chess.pgn.read_game(io.StringIO(file))
            X.append(pgn)
            y.append(get_ratings(pgn))
        return X, y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.pgn_files)





ModuleNotFoundError: No module named 'tensorflow'

In [186]:
pgn.headers['WhiteElo']

'2100'